In [ ]:
import time

config_file = "/workspace/D-FINE/configs/dfine/objects365/dfine_hgnetv2_x_obj2coco.yml"
checkpoint_file = "/workspace/dfine_checkpoints/dfine_x_obj2coco.pth"

IMAGE_NAME = "sagemaker_inference_server"
!docker build -t {IMAGE_NAME} .

In [ ]:
!docker rm -f sagemaker_inference_server_app 2>/dev/null || true

In [ ]:

CONTAINER_NAME = f"{IMAGE_NAME}_app"

docker_cmd = f"""
docker run --gpus=all --rm -m 8g -p 8080:8080 \\
  -e CONFIG_FILE={config_file} \\
  -e CHECKPOINT_FILE={checkpoint_file} \\
  -e DEVICE=cuda:0 \\
  --name {CONTAINER_NAME} \\
  {IMAGE_NAME}
"""

print("Running Docker command:\n", docker_cmd)
# !{docker_cmd}

In [ ]:
import requests
from test_inference_server import get_random_coco_images, get_plate_images

def make_local_request(image_url, model_type, model_enpoint = "http://localhost:8080/detect/"):
    response = requests.post(model_enpoint, json={"image_url": image_url, "model": model_type, "classes_to_detect": [0, 1, 2, 3, 4, 5]})
    if response.status_code != 200:
        print(f"Failed for {image_url}")
        return {}
    else:
        return response.json()

alpr_images = get_plate_images()
for image_url in alpr_images:
    print(f"Processing {image_url}")
    result = make_local_request(image_url, "license_plate_recognition")
    print(result)
    
coco_images = get_random_coco_images(5)
for image_url in coco_images:
    print(f"Processing {image_url}")
    result = make_local_request(image_url, "object_detection")
    print(result)